# MLP vs CNN
* Multi-Layer Perceptron
* Convolution Neural Network

## Preparation

### Load Library

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from utils.support_tf import LogLevelManager
LogLevelManager.set(2)

### Define Global Variables

In [2]:
EPOCHS = 5
BATCH_SIZE = 128

### Prepare Training Data

In [3]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

In [4]:
train_x, test_x = tf.expand_dims(train_x/255., -1), tf.expand_dims(test_x/255., -1)
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(train_x.shape[0]).batch(BATCH_SIZE)
# Test data are not shuffled.
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(BATCH_SIZE)

## MLP Model

### Define Using Class

In [18]:
class SimpleMLP(tf.keras.Model):
    
    def __init__(self):
        super(SimpleMLP, self).__init__(name="simple_nlp")
        
        self.i  = tf.keras.layers.Dense(units=64, activation="relu", input_shape=train_x.shape[1:])
        self.d1 = tf.keras.layers.Dense(units=128, activation="relu")
        self.d2 = tf.keras.layers.Dense(units=256, activation="relu")
        self.f1 = tf.keras.layers.Flatten()
        self.d3 = tf.keras.layers.Dense(units=256, activation="relu")
        self.o  = tf.keras.layers.Dense(units=10, activation="softmax")
        
    def call(self, inputs):
        l = self.i(inputs)
        l = self.d1(l)
        l = self.d2(l)
        l = self.f1(l)
        l = self.d3(l)
        l = self.o(l)
        return l

In [19]:
mlp = SimpleMLP()
mlp.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

### Training & Validation

In [20]:
mlp.fit(train_ds, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=test_ds)

Epoch 1/5
469/469 [==============================] - 12s 25ms/step - loss: 0.2430 - accuracy: 0.9273 - val_loss: 0.1144 - val_accuracy: 0.9651
Epoch 2/5
469/469 [==============================] - 12s 24ms/step - loss: 0.0996 - accuracy: 0.9706 - val_loss: 0.0972 - val_accuracy: 0.9715
Epoch 3/5
469/469 [==============================] - 11s 24ms/step - loss: 0.0652 - accuracy: 0.9797 - val_loss: 0.0856 - val_accuracy: 0.9736
Epoch 4/5
469/469 [==============================] - 12s 24ms/step - loss: 0.0478 - accuracy: 0.9842 - val_loss: 0.0799 - val_accuracy: 0.9769
Epoch 5/5
469/469 [==============================] - 12s 24ms/step - loss: 0.0372 - accuracy: 0.9876 - val_loss: 0.0873 - val_accuracy: 0.9738


### Evaluation

In [21]:
mlp.evaluate(test_ds)

79/79 [==============================] - 1s 12ms/step - loss: 0.0873 - accuracy: 0.9738


[0.08733433485031128, 0.973800003528595]

## CNN Model

### Define Using Class

In [35]:
class SimpleCNN(tf.keras.Model):
    
    def __init__(self):
        super(SimpleCNN, self).__init__(name="simple_cnn")
        self.ci = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), 
                                        #  strides=(3,3), padding="same",
                                         activation="relu", input_shape=train_x.shape[1:])
        self.mi = tf.keras.layers.MaxPooling2D(pool_size=(2,2))
        self.c1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), 
                                        #  strides=(3,3), padding="same",
                                         activation="relu")
        self.m1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))
        self.c2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), 
                                        #  strides=(3,3), padding="same",
                                         activation="relu")
        self.m2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))
        self.f  = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(units=256, activation="relu")
        self.o  = tf.keras.layers.Dense(units=10, activation="softmax")
        
    def call(self, inputs):
        l = self.ci(inputs)
        l = self.mi(l)
        l = self.c1(l)
        l = self.m1(l)
        l = self.c2(l)
        l = self.m2(l)
        l = self.f(l)
        l = self.d1(l)
        l = self.o(l)
        return l

In [36]:
cnn = SimpleCNN()
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

### Training & Validation

In [38]:
cnn.fit(train_ds, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=test_ds)

Epoch 1/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0198 - accuracy: 0.9940 - val_loss: 0.0462 - val_accuracy: 0.9858
Epoch 2/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0175 - accuracy: 0.9945 - val_loss: 0.0449 - val_accuracy: 0.9881
Epoch 3/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0152 - accuracy: 0.9948 - val_loss: 0.0433 - val_accuracy: 0.9887
Epoch 4/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0126 - accuracy: 0.9957 - val_loss: 0.0537 - val_accuracy: 0.9863
Epoch 5/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0108 - accuracy: 0.9964 - val_loss: 0.0868 - val_accuracy: 0.9819


### Evaluation

In [25]:
cnn.evaluate(test_ds)

79/79 [==============================] - 1s 7ms/step - loss: 0.0442 - accuracy: 0.9864


[0.04416196420788765, 0.9864000082015991]